In [1]:
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

# Input message
message = b"HI, MY NAME IS SANSKRITI"

# Generate RSA keys for encryption (as a Kyber substitute)
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

# Encrypt the message to simulate ciphertext generation
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# Display the ciphertext
print("Ciphertext:", ciphertext.hex())

# Derive shared key using HKDF for both sender and receiver
salt = os.urandom(16)  # Random salt
backend = default_backend()

# Sender's side shared key derivation
hkdf_sender = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    info=b"handshake data",
    backend=backend
)
shared_key_sender = hkdf_sender.derive(ciphertext)

# Receiver decrypts the message
decrypted_message = private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# Receiver's side shared key derivation
hkdf_receiver = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    info=b"handshake data",
    backend=backend
)
shared_key_receiver = hkdf_receiver.derive(ciphertext)

# Show the result
print("Input Message:", message.decode())
print("Ciphertext:", ciphertext.hex())
print("Shared Key (Sender):", shared_key_sender.hex())
print("Shared Key (Receiver):", shared_key_receiver.hex())
print("Key Match:", shared_key_sender == shared_key_receiver)


Ciphertext: 1be4bd1ec35288b64feaeeb238341b397c0ec64081a86aaa88eaafc1fb745341d6186c00f09916e8cc18687c570faa01d9511a22dfd1ca6fa00eba058094ce31e3460d249726e0e8c2f79f787440c1aa60b4e79323066cc1e3bda7c367006a0ec935249acbc94bbfd9b9739285a9e6eae7dc7486b25199d926b8ffe778b8b34e4fe7feb141abb48b190614acef6c42958c8d86f6ea97a0dc8823350f40ff722f94b93995b1aeee0817e23fbe24cc11bbef5ab7c00b8fb5c8c1756c3c94b48ab810aa88ae0043583b747f377e7570aeaf5400ef7d92f4aa9f3a2aa4db2bae9fc56a8e3839f55341882c8198c8943a0db3844258a63e99870a1cd0115f1ddacae9
Input Message: HI, MY NAME IS SANSKRITI
Ciphertext: 1be4bd1ec35288b64feaeeb238341b397c0ec64081a86aaa88eaafc1fb745341d6186c00f09916e8cc18687c570faa01d9511a22dfd1ca6fa00eba058094ce31e3460d249726e0e8c2f79f787440c1aa60b4e79323066cc1e3bda7c367006a0ec935249acbc94bbfd9b9739285a9e6eae7dc7486b25199d926b8ffe778b8b34e4fe7feb141abb48b190614acef6c42958c8d86f6ea97a0dc8823350f40ff722f94b93995b1aeee0817e23fbe24cc11bbef5ab7c00b8fb5c8c1756c3c94b48ab810aa88ae0043583b747f377e7570aeaf5400ef7